In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
import os
import csv
from skimage import io, transform
from PIL import Image

In [2]:
dataset_path = os.path.join(os.curdir, 'dataset')
with_mask_ds_path = os.path.join(dataset_path, 'aug', 'with_mask')
without_mask_ds_path = os.path.join(dataset_path, 'aug', 'without_mask')
csv_file = os.path.join(dataset_path, 'face-mask-detection-dataset.csv')

In [3]:
# Make a csv for dataset
if os.path.exists(csv_file):
    print("CSV File Found")
else:
    print("CSV Not Found")    
    print("Creating CSV File")
    with open(csv_file, mode='w') as f:
        col_names = ['image_name', 'label']
        writer = csv.DictWriter(f, fieldnames=col_names)
        writer.writeheader()
        for img in os.listdir(without_mask_ds_path):
            writer.writerow({'image_name': img, 'label': 0})
        for img in os.listdir(with_mask_ds_path):
            writer.writerow({'image_name': img, 'label': 1})
        f.close()
    print("CSV File Created")

CSV File Found


In [4]:
class MaskDetectionDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.csv_file = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.csv_file)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = self.csv_file.iloc[idx, 0]
        label = self.csv_file.iloc[idx, 1]
        if (label==0):
            img_path = os.path.join(self.root_dir, 'aug', 'without_mask', img_name)
            img = io.imread(img_path)
        else:
            img_path = os.path.join(self.root_dir, 'aug', 'with_mask', img_name)
            img = io.imread(img_path)
        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        sample = {
            'image': img,
            'label': label
        }
        return sample

In [5]:
augmentation_transform = transforms.Compose([
    transforms.Resize((320,320)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = MaskDetectionDataset(csv_file='./dataset/face-mask-detection-dataset.csv', root_dir=dataset_path, transform=augmentation_transform)

In [6]:
ds_len = len(dataset)
print(len(dataset))

5812


In [7]:
batch_size = 32

# Splitting dataset
train_set, test_set = torch.utils.data.random_split(dataset, [int(0.8*ds_len), ds_len-int(0.8*ds_len)])
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [8]:
for batch_idx, sample in enumerate(train_loader):
    print(batch_idx)
    print(sample['image'].shape)
    print(sample['label'].shape)
    break

0
torch.Size([32, 3, 320, 320])
torch.Size([32])


In [9]:
# Transfer Learning from MobileNetV2
model = torchvision.models.mobilenet_v2(pretrained=True)
# print(model)

In [10]:

for param in model.parameters():
    param.requires_grad = False


In [11]:
model.classifier[1] = nn.Sequential(
    nn.Linear(1280, 256),
    nn.ReLU(inplace=True),
    nn.Linear(256, 128),
    nn.ReLU(inplace=True),
    nn.Dropout(0.4),
    nn.Linear(128, 64),
    nn.ReLU(inplace=True),
    nn.Linear(64, 32),
    nn.ReLU(inplace=True),
    nn.Dropout(0.4),
    nn.Linear(32, 2),
)

In [12]:
#checking trainable parameters
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

371234


In [13]:
print(model.classifier)

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Sequential(
    (0): Linear(in_features=1280, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU(inplace=True)
    (4): Dropout(p=0.4, inplace=False)
    (5): Linear(in_features=128, out_features=64, bias=True)
    (6): ReLU(inplace=True)
    (7): Linear(in_features=64, out_features=32, bias=True)
    (8): ReLU(inplace=True)
    (9): Dropout(p=0.4, inplace=False)
    (10): Linear(in_features=32, out_features=2, bias=True)
  )
)


In [14]:
lr = 1e-4
epochs = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [15]:
# Training Data
from tqdm.notebook import tqdm

model.train()
model.to(device=device)
for epoch in range(epochs):
    num_samples = 0
    num_correct = 0
    running_loss = 0.0
    tk0 = tqdm(train_loader, total=int(len(train_loader)))
    for batch_idx, sample in enumerate(tk0):
        data = sample['image'].to(device=device)
        targets = sample['label'].to(device=device)
        
        pred_output = model(data)
        _, prediction = pred_output.max(1)
        num_samples += pred_output.size(0)
        num_correct += (prediction==targets).sum()
        
        loss = loss_function(pred_output, targets)
        running_loss += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        tk0.set_postfix(loss=(running_loss / (batch_idx * train_loader.batch_size)).item())
    print(f'Epoch #{epoch+1} completed with Accuracy {(num_correct / num_samples * 100.00):.2f}%')
print()
print("Training Completed!")


  0%|          | 0/146 [00:00<?, ?it/s]

Epoch #1 completed with Accuracy 84.25%


  0%|          | 0/146 [00:00<?, ?it/s]

Epoch #2 completed with Accuracy 97.85%


  0%|          | 0/146 [00:00<?, ?it/s]

...

Epoch #18 completed with Accuracy 99.46%


  0%|          | 0/146 [00:00<?, ?it/s]

Epoch #19 completed with Accuracy 99.29%


  0%|          | 0/146 [00:00<?, ?it/s]

Epoch #20 completed with Accuracy 99.46%

Training Completed!


In [16]:
# Model Testing
def check_accuracy(loader, model):
    model.eval() # Set model into evaluation mode!
    num_samples = 0
    num_correct = 0
    for batch_idx, sample in enumerate(loader):
        data = sample['image'].to(device=device)
        targets = sample['label'].to(device=device)
        
        pred_output = model(data)
        _, prediction = pred_output.max(1)
        num_samples += pred_output.size(0)
        num_correct += (prediction==targets).sum()
    
    print(f'Correctly identified samples: {num_correct}')
    print(f'Total samples: {num_samples}')
    print(f'The Validation Accuracy is {num_correct / num_samples * 100.00:.2f}')
    model.train() # Setting model back to training data

In [17]:
# print("Checking Accuracy on Training Data...")
# check_accuracy(train_loader, model)
# print()
print("Checking Accuracy on Testing Data...")
check_accuracy(test_loader, model)

Checking Accuracy on Testing Data...
Correctly identified samples: 1162
Total samples: 1163
The Validation Accuracy is 99.91


In [18]:
# Saving model with trained weights
torch.save(model.state_dict(), 'Trained Models/mask_detection_model.pth')

In [20]:
trained_model = torchvision.models.mobilenet_v2(pretrained=True)
trained_model.classifier[1] = nn.Sequential(
    nn.Linear(1280, 256),
    nn.ReLU(inplace=True),
    nn.Linear(256, 128),
    nn.ReLU(inplace=True),
    nn.Dropout(0.4),
    nn.Linear(128, 64),
    nn.ReLU(inplace=True),
    nn.Linear(64, 32),
    nn.ReLU(inplace=True),
    nn.Dropout(0.4),
    nn.Linear(32, 2),
)

trained_model.load_state_dict(torch.load('Trained Models/mask_detection_model.pth'))
trained_model.eval()

MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen

In [21]:
test_transform_compose = transforms.Compose([
    transforms.Resize((320,320)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [22]:
def prepare_image_for_testing(img):
    img = Image.fromarray(img)
    img = test_transform_compose(img)
    img = img.unsqueeze_(0)
    return img

In [23]:
without_mask_image = io.imread(os.path.join('Test Images', 'without_mask.jpg'))
without_mask_image = prepare_image_for_testing(without_mask_image)
without_mask_image = without_mask_image.to(device=device)
trained_model.to(device=device)
pred_output = trained_model(without_mask_image)
_, prediction = pred_output.max(1)
print("Label: 0")
if (prediction[0]==0):
    print("Prediction: 0")
else:
    print("Prediction: 1")

Label: 0
Prediction: 0


In [24]:
with_mask_image = io.imread(os.path.join('Test Images', 'with_mask.jpg'))
with_mask_image = prepare_image_for_testing(with_mask_image)
with_mask_image = with_mask_image.to(device=device)
trained_model.to(device=device)
pred_output = trained_model(with_mask_image)
_, prediction = pred_output.max(1)
print("Label: 1")
if (prediction[0]==0):
    print("Prediction: 0")
else:
    print("Prediction: 1")

Label: 1
Prediction: 1
